# Torch Vision Vision Transformers

https://arxiv.org/pdf/2010.11929v2.pdf

https://pytorch.org/vision/stable/models/vision_transformer.html

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

print("torchvision: {}.".format(torchvision.__version__))

torchvision: 0.13.1+cu113.


In [31]:
file_url = "https://upload.wikimedia.org/wikipedia/commons/2/23/Julia%2C_Japanese_porn_actress.jpg"

io.imsave("Downloaded_image.jpg", io.imread(file_url))
img = torchvision.io.read_image("Downloaded_image.jpg")

#s = 50
#img = img[:, s: s + 560, :]
print(img.shape)
#plt.imshow(img.permute(1, 2, 0))
#plt.show()

torch.Size([3, 900, 560])


In [28]:
# Step 1: Initialize model with the best available weights
weights = torchvision.models.ViT_B_16_Weights.IMAGENET1K_V1
model = torchvision.models.vit_b_16(weights = weights)

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

In [29]:
# Step 4: Use the model and print the predicted category
model.eval()
prediction = model(batch).squeeze(0).softmax(0)
prediction = prediction.detach().numpy()

class_ids = prediction.argsort()[-5:][::-1]
scores = prediction[class_ids]
category_names = np.array(weights.meta["categories"])[class_ids]

for i in range(len(category_names)):
    print("{}{} {:.2f}.".format(category_names[i], "." * (30 - len(category_names[i])), scores[i]))

bikini........................ 0.76.
brassiere..................... 0.09.
maillot....................... 0.05.
miniskirt..................... 0.00.
sarong........................ 0.00.


In [30]:
print(batch.shape)
x = model._process_input(batch)
print(x.shape)
n = x.shape[0]

batch_class_token = model.class_token.expand(n, -1, -1)
x = torch.cat([batch_class_token, x], dim = 1)
print(x.shape)

x = model.encoder(x)
print(x.shape)

x = x[:, 0]
print(x.shape)

x = model.heads(x)
print(x.shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 196, 768])
torch.Size([1, 197, 768])
torch.Size([1, 197, 768])
torch.Size([1, 768])
torch.Size([1, 1000])
